In [1]:
include("src/Trajectories.jl")
using .QuTaM

In [6]:
using LinearAlgebra

In [3]:
QuTaM.rf_params.psi0

2-element Vector{ComplexF64}:
 0.0 + 0.0im
 1.0 + 0.0im

In [15]:
q0 = norm(exp(-5im*params.multiplier*params.tf*sys.Heff)*params.psi0)

0.005876208123315946

In [11]:
params.multiplier*params.tf

10.0

In [2]:
# Create a data set of trajectories and evaluate them at given time
# Using rf as basis
sys = QuTaM.rf_sys
params = QuTaM.rf_params
data = QuTaM.run_trajectories(sys,params)

1000-element Vector{Vector{Main.QuTaM.DetectionClick}}:
 [Main.QuTaM.DetectionClick(2.220446049250313e-16, 1)]
 [Main.QuTaM.DetectionClick(0.04000800160032029, 1)]
 [Main.QuTaM.DetectionClick(0.1900380076015205, 1)]
 [Main.QuTaM.DetectionClick(0.41008201640328085, 1)]
 [Main.QuTaM.DetectionClick(0.1800360072014405, 1)]
 [Main.QuTaM.DetectionClick(0.020004000800160255, 1)]
 [Main.QuTaM.DetectionClick(0.0500100020004003, 1)]
 [Main.QuTaM.DetectionClick(0.03000600120024027, 1)]
 [Main.QuTaM.DetectionClick(0.03000600120024027, 1)]
 [Main.QuTaM.DetectionClick(0.08001600320064035, 1)]
 [Main.QuTaM.DetectionClick(0.03000600120024027, 1)]
 [Main.QuTaM.DetectionClick(0.010002000400080239, 1)]
 [Main.QuTaM.DetectionClick(0.1700340068013605, 1)]
 ⋮
 [Main.QuTaM.DetectionClick(0.08001600320064035, 1)]
 [Main.QuTaM.DetectionClick(0.020004000800160255, 1)]
 [Main.QuTaM.DetectionClick(0.08001600320064035, 1)]
 [Main.QuTaM.DetectionClick(0.0500100020004003, 1)]
 [Main.QuTaM.DetectionClick(0.0900180036